In [1]:
shift_requests = [[[1,1,0,1,1],[1,1,1,1,1],[1,1,0,1,1],[1,1,1,1,1],[1,1,0,1,1],[1,1,1,1,1],[1,1,1,1,1]],[[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0]],[[1,0,1,0,1],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1],[1,0,0,0,1]],[[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[0,1,1,0,0],[0,1,1,0,0]],[[0,1,1,1,1],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,1],[1,1,1,0,1],[0,1,1,1,1]],[[0,1,1,1,1],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,1],[1,1,1,0,1],[1,1,1,1,1]],[[0,1,1,0,0],[0,1,1,0,0],[0,1,1,0,0],[0,1,1,0,0],[0,1,1,0,0],[0,1,1,0,0],[0,1,1,0,0]],[[0,1,1,0,1],[0,1,1,0,1],[0,1,1,0,1],[0,1,1,0,1],[0,1,1,0,1],[0,1,1,0,1],[0,1,1,0,1]],[[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0]],[[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0]],[[0,1,1,1,1],[0,1,1,1,1],[0,1,1,0,1],[0,1,1,1,1],[0,1,1,1,1],[0,1,1,1,1],[0,1,1,1,1]],[[0,1,1,1,1],[0,1,1,1,1],[0,1,1,1,1],[0,1,1,1,1],[0,0,1,1,1],[1,1,1,0,1],[1,1,1,0,1]],[[1,0,0,1,0],[1,0,0,1,0],[1,0,0,1,0],[1,0,0,1,0],[1,0,0,1,0],[1,0,1,0,0],[1,0,1,0,0]],[[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1]],[[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[1,1,1,1,1],[1,1,1,1,1]],[[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0],[1,1,1,0,0]],[[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1],[0,0,0,1,1]],[[1,0,1,0,1],[1,0,1,0,1],[1,0,1,0,1],[1,0,1,0,1],[1,0,1,0,1],[1,0,1,0,1],[0,1,0,1,1]]]
names = ["phamhomaitran",
"thamtiin",
"Phi",
"Tam Nguyen",
"quyenphan",
"thusuong",
"linhnguyen",
"freeman",
"giangvu",
"ngocvo",
"lionelphong",
"hoangphuong",
"dangkhoa",
"kim",
"nguyenanh21",
"phantan",
"phuongtien",
"tranthuc"]
len(names)

18

In [10]:
from __future__ import print_function
import sys
from ortools.sat.python import cp_model

# This program tries to find an optimal assignment of customer supports to shifts
# (5 shifts per day, for 7 days), subject to some constraints (see below).
# Each supporter can request to be assigned to specific shifts.
# The optimal assignment maximizes the number of fulfilled shift requests.
num_supporters = len(names)
num_shifts = 5
num_days = 7
all_supporters = range(num_supporters)
all_shifts = range(num_shifts)
all_days = range(num_days)

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(n, d, s)]: supporter 'n' works shift 's' on day 'd'.
shifts = {}
for n in all_supporters:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d,
                    s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

# Each shift is assigned from 5 - 6 supporters
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)] for n in all_supporters) >= 5)
        model.Add(sum(shifts[(n, d, s)] for n in all_supporters) <= 6)

# Each supporter works at most two shifts per day.
for n in all_supporters:
    for d in all_days:
        model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 2)

# works_shift[(n, d)] is 1 if supporter n works  at least one shift in
# that day.
works_day = {}
for n in all_supporters:
  for d in all_days:
    works_day[(n, d)] = model.NewBoolVar('works_day_n%id%i' % (n, d))
    model.AddMaxEquality(works_day[(n, d)],
                         [shifts[(n, d, s)] for s in all_shifts])
for n in all_supporters:
    model.Add(sum(works_day[(n, d)] for d in all_days) == num_days - 1)

# No supporter works two continuous shifts
for n in all_supporters:
    for d in all_days:
        for s in all_shifts:
            next_s = (s + 1) % num_shifts
            if next_s == 0:
                next_day = (d + 1) % num_days
                next_shift = shifts[(n, next_day, next_s)]
            else:
                next_shift = shifts[(n, d, next_s)]
            model.Add((shifts[(n, d, s)] + next_shift) < 2)

# min_shifts_assigned is the largest integer such that every supporter can be
# assigned at least that number of shifts.
min_shifts_per_supporter = 9
max_shifts_per_supporter = min_shifts_per_supporter + 1
for n in all_supporters:
    num_shifts_worked = sum(
        shifts[(n, d, s)] for d in all_days for s in all_shifts)
    model.Add(min_shifts_per_supporter <= num_shifts_worked)
    model.Add(num_shifts_worked <= max_shifts_per_supporter)

model.Maximize(
    sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_supporters
        for d in all_days for s in all_shifts))
# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 8

status = solver.Solve(model)
print(status)
for d in all_days:
    print('Day', d)
    for n in all_supporters:
        for s in all_shifts:
            if solver.Value(shifts[(n, d, s)]) == 1:
                if shift_requests[n][d][s] == 1:
                    print('Supporter', names[n], 'works shift', s, '(requested).')
                else:
                    print('Supporter', names[n], 'works shift', s, '(not requested).')
    print()

# Statistics.
print()
print('Statistics')
print('  - wall time       : %f s' % solver.WallTime())


4
Day 0
Supporter phamhomaitran works shift 0 (requested).
Supporter phamhomaitran works shift 3 (requested).
Supporter thamtiin works shift 0 (requested).
Supporter Phi works shift 2 (requested).
Supporter Tam Nguyen works shift 0 (requested).
Supporter Tam Nguyen works shift 2 (requested).
Supporter quyenphan works shift 1 (requested).
Supporter quyenphan works shift 3 (requested).
Supporter thusuong works shift 2 (requested).
Supporter linhnguyen works shift 1 (requested).
Supporter linhnguyen works shift 4 (not requested).
Supporter giangvu works shift 1 (requested).
Supporter giangvu works shift 3 (requested).
Supporter ngocvo works shift 1 (requested).
Supporter ngocvo works shift 3 (requested).
Supporter lionelphong works shift 2 (requested).
Supporter lionelphong works shift 4 (requested).
Supporter hoangphuong works shift 1 (requested).
Supporter hoangphuong works shift 4 (requested).
Supporter dangkhoa works shift 0 (requested).
Supporter dangkhoa works shift 3 (requested).
S

In [11]:
import pandas as pd

table = []
for d in all_days:
    print('Day', d)
    data = []
    for s in all_shifts:      
        shift_data = []
        for n in all_nurses:
            if solver.Value(shifts[(n, d, s)]) == 1:
                if shift_requests[n][d][s] == 1:
                    shift_data.append(names[n])
                else:
                    shift_data.append(names[n] + '(*)')
        data.append(shift_data)
    df = pd.DataFrame(data).transpose()
    print(df.to_string(index=False))
    table.append(data)
    
table_df = pd.DataFrame(table).transpose().to_csv("output.csv", index=False)

Day 0
             0            1               2              3              4
 phamhomaitran    quyenphan             Phi  phamhomaitran  linhnguyen(*)
      thamtiin   linhnguyen      Tam Nguyen      quyenphan    lionelphong
    Tam Nguyen      giangvu        thusuong        giangvu    hoangphuong
      dangkhoa       ngocvo     lionelphong         ngocvo    nguyenanh21
      tranthuc  hoangphuong  nguyenanh21(*)       dangkhoa       tranthuc
Day 1
        0            1              2            3            4
 thamtiin    quyenphan       thamtiin       Phi(*)      freeman
      Phi     thusuong        freeman     thusuong  lionelphong
 dangkhoa   linhnguyen         ngocvo      giangvu          kim
   kim(*)  lionelphong        phantan  hoangphuong   phuongtien
  phantan  hoangphuong  phuongtien(*)     dangkhoa     tranthuc
Day 2
              0           1         2            3              4
  phamhomaitran  Tam Nguyen  thamtiin   Tam Nguyen            Phi
            Phi   quye

In [48]:
table = []
data = []
for d in all_days:
    print('Day', d)
    for s in all_shifts:      
        shift_data = []
        for n in all_nurses:
            if solver.Value(shifts[(n, d, s)]) == 1:
                if shift_requests[n][d][s] == 1:
                    shift_data.append(names[n])
                else:
                    shift_data.append(names[n] + '(*)')
        data.append(shift_data)

df = pd.DataFrame(data).transpose().to_csv("output1.csv", index=False)



Day 0
Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
